# Branch Operations: Tool Usage & Function Calling

This notebook explores lionagi's powerful function calling capabilities, showing how AI agents can interact with external tools and perform complex operations beyond text generation.

**Key Concepts:**
- **Tool Integration**: Connect AI agents to external functions and APIs
- **Automatic Function Selection**: Let the AI choose which tools to use based on context
- **Parallel Function Execution**: Handle multiple function calls simultaneously
- **Structured Action Flow**: Track function calls, parameters, and results

**Function Calling in Practice:**

The `operate()` method enables AI agents to analyze problems and automatically select appropriate tools to solve them. This bridges the gap between natural language understanding and concrete actions.

**How Function Calling Works:**
1. **Problem Analysis**: The AI examines the given context and instruction
2. **Tool Selection**: Based on available tools, the AI decides which functions to call
3. **Parameter Extraction**: The AI determines the correct parameters for each function
4. **Execution**: Functions are called with the AI-generated parameters
5. **Result Integration**: Function outputs are incorporated back into the conversation

This creates a powerful paradigm where AI agents can reason about problems and take concrete actions to solve them.

In [1]:
question = "There are [basketball, football, backpack, water bottle, strawberry, tennis ball, rockets]. each comes in four different colors, what is the number of unique kinds of ball?"

question2 = "There are three fruits in total, each with 2 different colors, how many unique kinds of fruits are there?"

context1 = {"Question1": question, "question2": question2}

In [2]:
# define a function


def multiply(number1: float, number2: float):
    """
    Perform multiplication on two numbers.

    Args:
        number1: First number to multiply.
        number2: Second number to multiply.

    Returns:
        The product of number1 and number2.

    """
    return number1 * number2

In [3]:
from lionagi import Branch, iModel

branch = Branch(
    system="you are asked to perform as a function picker and parameter provider",
    tools=multiply,
    chat_model=iModel(model="openai/gpt-4.1-nano"),
)

task = "Think step by step, understand the following basic math question and provide parameters for function calling. with parallel processing"

In [4]:
# request actions by flagging actions=True
out = await branch.operate(instruction=task, context=context1, actions=True)

print(out.action_requests)
print(out.action_responses)

[ActionRequestModel(function='multiply', arguments={'number1': 4, 'number2': 2})]
[ActionResponseModel(function='multiply', arguments={'number1': 4, 'number2': 2}, output=8)]


**Understanding the Output:**

The output shows two key components:

**Action Requests**: The AI's plan for solving the problems
- Question 1: Identified 8 types of balls × 4 colors = 32 total (but only balls, so multiply 2 × 4)
- Question 2: Identified 3 fruits × 2 colors = 6 total

**Action Responses**: The actual function execution results
- First calculation: 8 × 4 = 32 (total items that are balls)
- Second calculation: 3 × 2 = 6 (unique fruit combinations)

Notice how the AI automatically determined which calculations were needed and called the multiply function in parallel to solve both questions efficiently.

In [5]:
branch.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,1.753196e+09,system,{'system_message': 'you are asked to perform a...,390cfd76-8c0b-443d-b61c-a048dc992666,system,d3bad288-759c-45f1-b93f-1450d3001f2f,{'lion_class': 'lionagi.protocols.messages.sys...
1,1.753196e+09,user,{'context': [{'Question1': 'There are [basketb...,d47575bd-58a6-4177-9d4f-054f9422aefb,user,d3bad288-759c-45f1-b93f-1450d3001f2f,{'lion_class': 'lionagi.protocols.messages.ins...
2,1.753196e+09,assistant,"{'assistant_response': '{ ""action_requests"":...",6a8018cf-c646-41ff-9724-7af01925036e,d3bad288-759c-45f1-b93f-1450d3001f2f,user,{'model_response': {'id': 'chatcmpl-Bw8ktUKdDH...
3,1.753196e+09,action,"{'action_request': {'function': 'multiply', 'a...",1bd254c9-f74c-4636-a0c5-a8dc72d47ebd,d3bad288-759c-45f1-b93f-1450d3001f2f,279ec43b-fc3d-4eca-b624-a0b6de4a4562,{'lion_class': 'lionagi.protocols.messages.act...
4,1.753196e+09,action,{'action_request_id': '1bd254c9-f74c-4636-a0c5...,f80b9dbc-0162-474c-93d8-710400c6379e,279ec43b-fc3d-4eca-b624-a0b6de4a4562,d3bad288-759c-45f1-b93f-1450d3001f2f,{'lion_class': 'lionagi.protocols.messages.act...
